# 前情提要
此次作業交的 CSV 檔為 TF-IDF + Logistic Regression 的結果  
在 Kaggle 的成績為 0.70750

In [ ]:
# 要預先載入
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

[nltk 教學](https://clay-atlas.com/blog/2019/07/30/%E3%80%90nlp%E3%80%91nltk-%E5%9F%BA%E6%9C%AC%E6%95%99%E5%AD%B8/)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
corpus_root = 'drive/My Drive/DS HW/' 

df = pd.read_csv(corpus_root+'train.csv')
df_test = pd.read_csv(corpus_root+'test_no_answer.csv')

## 前處理

### 刪除標點符號  

ref: https://gist.github.com/youngsoul/34d4ded8d4edb82fdfb8f9a91ade2c0d  

    table = str.maketrans({key: None for key in string.punctuation})
    return text.lower().translate(table)
ref: https://www.kite.com/python/answers/how-to-remove-all-punctuation-marks-with-nltk-in-python  

ref: http://blog.fantasy.codes/python/2013/07/02/py-re-module/


In [ ]:
import re
import string

# 還原縮寫 主要還原 not
abbr = {        
    "isn t ":"is not ",
    "wasn t ":"was not ",
    "aren t ":"are not ",
    "weren t ":"were not ",
    "can t ":"can not ",
    "couldn t ":"could not ",
    "don t ":"do not ",
    "doesn t ":"does not ",
    "didn t ":"did not ",
    "won t ":"will not ",
    "wouldn t ":"would not ",
    "hasn t ":"has not ",
    "haven t ":"have not ",
    "hadn t ":"had not ",
    "shouldn t ":"should not ",
}

def remove_punctuation(text):
    tokenizer = nltk.RegexpTokenizer(r'\w+')
    new_words = tokenizer.tokenize(text)
    new_words = ' '.join(new_words)
    
    for word, origin in abbr.items():
        new_words = new_words.replace(word, origin)

    return new_words

### 詞型還原  

ref: https://www.zmonster.me/2016/01/21/lemmatization-survey.html





In [ ]:
from nltk.corpus import wordnet, stopwords
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else :
        return  None

def lemmatize_sentence(sentence):
    res = []
    lemmatizer = WordNetLemmatizer()
    for word, pos in pos_tag(word_tokenize(sentence)):
        wordnet_pos = get_wordnet_pos(pos) or wordnet.NOUN
        res.append(lemmatizer.lemmatize(word, pos=wordnet_pos))

    return ' '.join(res)

## TF-IDF
[min_df and max_df](https://stackoverflow.com/questions/27697766/understanding-min-df-and-max-df-in-scikit-countvectorizer) [.](https://intellipaat.com/community/3371/understanding-mindf-and-maxdf-in-scikit-countvectorizer)

[依比例切分Postive & Negative 的資料當 Training Data](https://zhuanlan.zhihu.com/p/35919812)

[pandas concatenating 合併資料 <concat 設定 axis=0 為直向合併](http://violin-tao.blogspot.com/2017/06/pandas-2-concat-merge.html)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

max_features = 50000

df['TEXT'] = df['TEXT'].apply(remove_punctuation)
df['TEXT'] = df['TEXT'].apply(lemmatize_sentence)
df_test['TEXT'] = df_test['TEXT'].apply(remove_punctuation)
df_test['TEXT'] = df_test['TEXT'].apply(lemmatize_sentence)
print(df['TEXT'])

# https://intellipaat.com/community/3371/understanding-mindf-and-maxdf-in-scikit-countvectorizer
tfidf = TfidfVectorizer(max_features=max_features, ngram_range=(1, 2), sublinear_tf = True, norm='l2')

df['TFIDF'] = list(tfidf.fit_transform(df['TEXT']).toarray())
df_test['TFIDF'] = list(tfidf.transform(df_test['TEXT']).toarray())

X_train = list(df['TFIDF'])
y_train = list(df["LABEL"])

0         good to know if you can not find these elsewhere
1        love it the grill plat come out and pop in the...
2        i m convince this be a poorly execute refurbis...
3        i would never have complain about that if it w...
4        the photo show the same whole large candy we v...
                               ...                        
10995                 i do not quite get it the first time
10996    i ve try instal with and without the oem stock...
10997    i be park at a truck stop in the cincinnati oh...
10998    i recently buy this case after see some youtub...
10999    the keyboard type only of the time and the mou...
Name: TEXT, Length: 11000, dtype: object


## SVM
TF-IDF + SVM

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.svm import LinearSVC, SVC

svm_model = LinearSVC()
svm_model.fit(X_train, y_train)

svm_pred_test = svm_model.predict(list(df_test['TFIDF']))
print(svm_pred_test)

[1 0 1 ... 0 0 0]


## BaseLine輸出檔案
TF-IDF + SVM

In [ ]:
Bsubmission = pd.DataFrame({'row_id': df_test['row_id'],
                           'LABEL': svm_pred_test})
Bsubmission.to_csv(corpus_root + 'BaseLineSubmission.csv', index = False)
Bsubmission

,row_id,LABEL
0,0,1
1,1,0
2,2,1
3,3,0
4,4,0
...,...,...
1995,1995,0
1996,1996,0
1997,1997,0
1998,1998,0
